In [1]:
from __future__ import print_function
import rospy
# Brings in the SimpleActionClient
import actionlib
import actionlib.msg
# Brings in the messages used by the Planning action including the goal message and the result message.
import assignment_2_2023.msg
from assignment_2_2023.msg import RobotState, PlanningFeedback, PlanningGoal, PlanningAction, PlanningResult 
from geometry_msgs.msg import Twist, Point, Pose
from actionlib_msgs.msg import GoalStatus
from nav_msgs.msg import Odometry
# libraries the UI
from ipywidgets import VBox,HBox
import ipywidgets as widgets
import jupyros as jr
from IPython.display import display, clear_output
from colorama import Fore, Style
from colorama import init
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt


In [2]:
class Visualiser:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro' ,label='Current Position')
        self.sent_goals, =self.ax.plot([],[],'bo',label='Reached Goals')
        self.canceled_goals, = self.ax.plot([], [], 'yo',label='Canceled Goals') 
        self.x_data, self.y_data = [] , [] 
        self.latest_position_x, self.latest_position_y = [],[]
        self.target_x, self.target_y = [], []
        self.canceled_x, self.canceled_y = [],[]
      
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.legend()  
     


    def odom_callback(self, odom_msg):
        x = odom_msg.pose.pose.position.x
        y = odom_msg.pose.pose.position.y
        vel_x = odom_msg.twist.twist.linear.x
        vel_z = odom_msg.twist.twist.angular.z
        self.latest_position_x.append(x)
        self.latest_position_y.append(y)
        
        global postion_label
        position_label.value = f"Position: ({self.latest_position_x:.2f}, {self.latest_position_y:.2f})"
         
        
    def update_plot(self, frame):
        self.ln.set_data( self.latest_position_x, self.latest_position_y)
        self.sent_goals.set_data(self.target_x,self.target_y)
        self.canceled_goals.set_data(self.canceled_x, self.canceled_y)
        return self.ln, self.sent_goals,  self.canceled_goals

        

In [3]:
#create a user interface node 
rospy.init_node('user_interface')
%matplotlib widget
vis = Visualiser()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# create the action client 

client = actionlib.SimpleActionClient('/reaching_goal',PlanningAction)
client.wait_for_server()

True

In [5]:
# Widget for setting goals
x_input = widgets.FloatText(description='X:')
y_input = widgets.FloatText(description='Y :')
send_goal_button = widgets.Button(description='Send Goal')
cancel_goal_button = widgets.Button(description='Cancel Goal')
position_label = widgets.Label(value="Position: (0, 0)")



In [6]:

coord = (0,0)
#function for sending the goal 
def send_goal(e):
    
    global coord
    print("Goal sent")
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x_input.value
    goal.target_pose.pose.position.y = y_input.value
    client.send_goal(goal,goal_done_callback)
   
    coord = (x_input.value, y_input.value)
   
  

# Bind the button's on_click event to the send_goal function
send_goal_button.on_click(send_goal)

In [7]:
goals_reached = []
coord = (0,0)
def goal_done_callback(status, result):
    global goals_reached, coord
    coord
    if status == actionlib.GoalStatus.SUCCEEDED:
        print("Goal succeeded.\n")
        vis.target_x.append(coord[0])
        vis.target_y.append(coord[1])
    else:
        print("Goal failed.\n")
        vis.canceled_x.append(coord[0])
        vis.canceled_y.append(coord[1])


In [8]:
canceled_target_array = []
#function to cancel the goal 
def cancel_goal(e):
    print("canceling the goal")
    client.cancel_goal()
    
    cancel_state = client.wait_for_result()

    if cancel_state:
        print("Goal has been cancelled.")
    else:
        print("Failed to cancel the goal")
    
    global canceled_target_array
    canceled_target_array.append(coord)

#bind the button on_click event to cancel the goal 
cancel_goal_button.on_click(cancel_goal)

In [9]:

ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)
plt.show(block=True)



In [10]:
odom_sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)

In [11]:
# Display the widgets
display(VBox([x_input, y_input, send_goal_button, cancel_goal_button,position_label]))

#keep the ros node running 
#rospy.spin()


Goal sent
